In [ ]:
import pandas as pd

In [ ]:
backup_date='2017-6-9'

In [ ]:
ratings = pd.read_csv('DB-backup/{}/Rating.csv'.format(backup_date),header=None)
users = pd.read_csv('DB-backup/{}/Users.csv'.format(backup_date),header=None)
v2s = pd.read_csv('DB-backup/{}/VideoRoadSeg.csv'.format(backup_date),header=None)
seg = pd.read_csv('DB-backup/{}/RoadSegment.csv'.format(backup_date),header=None)
login = pd.read_csv('DB-backup/{}/loginLog.csv'.format(backup_date),header=None)

In [ ]:
seg = seg.sort_values('index_seg')
def get_sql(row):
    return 'INSERT INTO `RoadSegment`(`sid`, `segmentid`, `index_seg`, `sumScore`, `sumRatio`, `sumCnt`, `geometry`) VALUES ({},{},{},{},{},{},"{}");'.format(row.sid, row.segmentid, row.index_seg, row.sumScore, row.sumRatio, row.sumCnt, row.geometry)

# with open('RoadSegment.sql', 'w') as f:
#     for idx, row in seg.iterrows():
#         f.write(get_sql(row)+'\n')

In [ ]:
ratings.columns = ['rid', 'uid','email','vid','rating','comment','tags','familiar','UTCts','timezone','watched','interaction','lid']
v2s.columns = ['vrsid','vid','sid','clip_name','index_seg','ratio',]
seg.columns=['sid','segmentid','index_seg','sumScore','sumRatio','sumCnt','geometry',]

In [ ]:
users_in_ratings = ratings.groupby('uid').nunique()
# unique vids per user 
users_in_ratings['vid'].value_counts()

In [ ]:
r_on_v = ratings.groupby('vid')['rating'].agg(['sum','count']).reset_index()

In [ ]:
s_from_r = r_on_v.merge(v2s)[['sid', 'count', 'sum', 'ratio']]

In [ ]:
s_from_r['total_ratio'] = s_from_r['count'] * s_from_r.ratio
s_from_r['total_sum'] = s_from_r['sum'] * s_from_r.ratio

In [ ]:
s2s = s_from_r.groupby('sid')[['total_sum','count', 'total_ratio']].sum().reset_index().merge(seg)

In [ ]:
s2s.shape